### <center><font color=navy> Tutorial #4 Computer- and robot-assisted surgery</font></center>
## <center><font color=navy> Tomography II</font></center>
<center>&copy; Sebastian Bodenstedt, National Center for Tumor Diseases (NCT) Dresden<br>
    <a href="https://www.nct-dresden.de/"><img src="https://www.nct-dresden.de/++theme++nct/images/logo-nct-en.svg"></a> </center>

## <center><font color=navy>Preperation</font></center>

For this tutorial, we will utilize the OpenCV, Matplotlib and NumPy:

In [ ]:
# Install numpy, opencv and matplot-lib pip packages into the current Jupyter kernel
import sys
!{sys.executable} -m pip install numpy opencv-python matplotlib matplotlib-inline

In [ ]:
import cv2
import numpy as np
# Force Matplotlib to display data directly in Jupyter
%matplotlib inline 
from matplotlib import pyplot as plt
import scipy.interpolate
from scipy import ndimage
import math

We will also download and extract a few image sequences:

In [ ]:
import urllib.request
from os.path import basename, exists
import zipfile

def download_and_extract(url): #download and extract Zip archive
    file_path = basename(url)
    if not exists(file_path): # does zip file already exist?
        urllib.request.urlretrieve(url, file_path) # if not, download it
        with zipfile.ZipFile(file_path, 'r') as zip_ref: # and unzip it
            zip_ref.extractall(".")

In [ ]:
download_and_extract("http://tso.ukdd.de/crs/TomographyII.zip") # In case you didn't download the data last week

We now list the extracted files:

In [ ]:
!dir *

## <center><font color=navy>Basic functions</font></center>
We define a function for displaying grayscale images

In [ ]:
def show_gray(img, canvas=plt, title=""): # Later we want to draw on a different underground, so we define this as a parameter
    canvas.imshow(img, cmap='gray', vmin=0, vmax=255)
    if not title == "":
        canvas.set_title(title)

## <center><font color=navy>Review image loading and rotation</font></center>
First, we develop a method to rotate 2D images:

In [ ]:
# Load image in grayscale
#img_gray = cv2.imread("Exercise2/Example.jpg", cv2.IMREAD_GRAYSCALE)
#img_gray = cv2.imread("Exercise3/img_01_raw.png", cv2.IMREAD_GRAYSCALE)
img_gray = cv2.imread("TomographyII/maulwurf.jpg", cv2.IMREAD_GRAYSCALE)

# Make sure image is square
size = min(img_gray.shape[0],img_gray.shape[1])
img_gray = img_gray[:size,:size]

figure, axis = plt.subplots(1, 2, figsize=(15, 15)) # subplots let you visualize multiple outputs simultanously

show_gray(img_gray, axis[0])

img_rot2 = ndimage.rotate(img_gray, 45)

show_gray(img_rot2, axis[1])
print(img_gray.shape, img_rot2.shape)

In [ ]:
figure, axis = plt.subplots(1, 4, figsize=(15, 15)) # subplots let you visualize multiple outputs simultanously

show_gray(img_gray, axis[0])

shape = img_gray.shape

img_rot = ndimage.rotate(img_gray, 45)

show_gray(img_rot, axis[1])

img_rot_back = ndimage.rotate(img_rot, -45)

show_gray(img_rot_back, axis[2])

offset_y = (img_rot_back.shape[0] - shape[0])//2 # Find Center of image
offset_x = (img_rot_back.shape[1] - shape[1])//2
         
img_rot_back_crop = img_rot_back[offset_y:offset_y + shape[0],offset_x:offset_x + shape[0]] # Crop result and apply it to output
show_gray(img_rot_back_crop, axis[3])        

## <center><font color=navy>Fourier Transformation in NumPy</font></center>
Reminder, we can represent an image by the frequencies it contains, using the Fourier Transformation:

In [ ]:
ft = np.fft.fft2(img_gray) # 2D fourier transform of our image
ft_centered = np.fft.fftshift(ft) # shift the transform, so the origin is in the center of the image

We can visualize the spectrum of the image

In [ ]:
spec = np.log(abs(ft_centered))

And we can also reverse the Fourier Transform

In [ ]:
ift = np.fft.ifftshift(ft_centered) # Reverse shift
ift = np.fft.ifft2(ift) # Inverse Fourier

In [ ]:
figure, axis = plt.subplots(1, 3, figsize=(15, 15)) # subplots let you visualize multiple outputs simultanously
axis[0].imshow(img_gray, cmap='gray')
axis[1].imshow(spec, cmap='gray')
axis[2].imshow(np.real(ift), cmap='gray')


You can modify the spectrum using image manipulation. One way is to create a mask and apply it to the spectrum:

In [ ]:
radius = 50

mask = np.zeros(ft_centered.shape, dtype=np.uint8) # Create an empty image
mask = cv2.circle(mask, (ft_centered.shape[0]//2, ft_centered.shape[0]//2), radius,1,-1) # Draw a circle
plt.imshow(mask, cmap='gray')

In [ ]:
ft_centered_mod = ft_centered.copy()
ft_centered_mod[mask == 0] = 0 # set frequencies outside circle to 0

spec_mod = np.log(abs(ft_centered_mod))
figure, axis = plt.subplots(1, 2, figsize=(15, 15)) # subplots let you visualize multiple outputs simultanously
axis[0].imshow(spec, cmap='gray')
axis[1].imshow(spec_mod, cmap='gray')

In [ ]:
ift = np.fft.ifftshift(ft_centered_mod)
ift = np.fft.ifft2(ift)

figure, axis = plt.subplots(1, 2, figsize=(15, 15)) # subplots let you visualize multiple outputs simultanously
show_gray(img_gray, axis[0])
axis[1].imshow(np.real(ift), cmap='gray')
print(np.min(np.real(ift)), np.max(np.real(ift)), np.mean(np.real(ift)))

## <center><font color=navy>Simulated CT</font></center>

Next, we simulate the projection step of a CT:

In [ ]:
def calc_proj(image, step_size):
    assert image.shape[0] == image.shape[1] # check if images are square
    num_steps = 180//step_size # Calculate the number of steps
    diag = round(math.sqrt(2*image.shape[0]*image.shape[0])) # calculate diagonal of the image, i.e. the maximum number of pixels when rotated 45 degrees
    print(diag)
    
    projections = np.zeros((num_steps, diag), dtype=np.float64) # Setup container for all projections
    
    count = 0
    for angle in range(0, 180, step_size):
        rot_image = ndimage.rotate(image, angle) # Rotate image
        proj = np.sum(rot_image, axis=0) # Calculate projection by adding values along axis 0
        
        projections[count, 0:rot_image.shape[0]] = proj # save projection
        count+=1
    
    return projections

In [ ]:
degrees = 1 # Step size
projs = calc_proj(img_gray, degrees) # Calculate projections
shape = img_gray.shape
print(projs.shape)
data = {"projections" : projs, "shape" : shape, "step_size" : degrees}

In [ ]:
import pickle

pickle.dump( data, open( "TomographyII/output.pkl", "wb" ) ) #Save projections

## <center><font color=navy>Reconstruction with Fourier</font></center>
First, we load the mystery image from last time that was encoded with the code above

In [ ]:
data = pickle.load( open( "TomographyII/output.pkl", "rb" ) ) # Load a secret

step_size = data["step_size"]
projs = data["projections"]
shape = data["shape"]

print(step_size)

Now, we will reconstruct the image using the Radon Transformation/Fourier Slice Theorem

In [ ]:
def reconstruction(input_proj, step_size, output_shape):
    """
    Reconstructs an image from CT projections using the back-projection method.
    
    Parameters:
    - input_proj: numpy array containing the projections (shape: num_projections x projection_length).
    - step_size: int, angle step size in degrees for each projection.
    - output_shape: tuple, shape of the output 2D reconstructed image.
    
    Returns:
    - 2D numpy array representing the reconstructed image in the Fourier domain.
    """
    # Initialize the output image as a complex array
    output = np.zeros(output_shape, dtype=np.complex64)
    # Create an empty mask for rotation purposes (not used in computation but for reference)
    mask = np.zeros(output_shape, dtype=np.int64)

    # Process each projection at the specified angle increment
    for count, angle in enumerate(range(0, 180, step_size)):
        # Rotate the mask to the current angle (used for determining rotation size)
        rotated_mask = scipy.ndimage.rotate(mask, angle)
        # Create a blank rotated image with the same shape as the rotated mask
        rotated_image = np.zeros(rotated_mask.shape, dtype=np.complex64)

        # Extract the current projection and apply the 1D Fourier Transform
        projection = input_proj[count, :rotated_image.shape[0]]
        ft_projection = np.fft.fftshift(np.fft.fft(np.fft.ifftshift(projection)))

        # Place the Fourier transform result in the center row of the rotated image
        rotated_image[rotated_image.shape[0] // 2] = ft_projection

        # Rotate the image back to its original orientation
        rotated_back_image = ndimage.rotate(rotated_image, -angle)

        # Calculate the offset for cropping the rotated image to match the output shape
        offset_y = (rotated_back_image.shape[0] - output_shape[0]) // 2
        offset_x = (rotated_back_image.shape[1] - output_shape[1]) // 2

        # Crop the image and add it to the output
        output += rotated_back_image[offset_y:offset_y + output_shape[0], offset_x:offset_x + output_shape[1]]

    return output

In [ ]:
res = reconstruction(projs, step_size, shape)

In [ ]:
res2 = np.real(np.fft.fftshift(np.fft.ifft2(np.fft.ifftshift(res))))

plt.imshow(res2, cmap='gray')

In [ ]:
spec_mod = np.log(abs(res))
figure, axis = plt.subplots(1, 2, figsize=(15, 15)) # subplots let you visualize multiple outputs simultanously

axis[0].imshow(spec_mod, cmap='gray')
axis[1].imshow(spec, cmap='gray')

In [ ]:
def reconstruction2(input_proj, step_size, output_shape):
    """
    Reconstructs a 2D Fourier Transform from CT projections.
    
    Parameters:
    - input_proj: numpy array of shape (num_projections, projection_length) containing CT projections.
    - step_size: int, angle step size in degrees for each projection.
    - output_shape: tuple, shape of the output 2D reconstructed image.
    
    Returns:
    - 2D numpy array representing the reconstructed Fourier Transform.
    """
    # Generate an empty mask of ones for rotation reference
    mask = np.ones(output_shape, dtype=np.int64)

    # Initialize lists to store source coordinates and Fourier transform values
    src_x_coords = []
    src_y_coords = []
    fft_values = []

    # Process each projection at the specified step size
    for idx, angle in enumerate(range(0, 180, step_size)):
        # Convert the current angle to radians
        rad_angle = np.deg2rad(angle)

        # Rotate the mask to align with the current angle (used for coordinate mapping)
        rotated_mask = scipy.ndimage.rotate(mask, angle)
        num_points = rotated_mask.shape[0]

        # Calculate the source coordinates for the rotated mask
        center_offset = (np.arange(num_points) - num_points / 2) / num_points * output_shape[0]
        src_x = (output_shape[0] / 2) + center_offset * np.cos(rad_angle)
        src_y = (output_shape[0] / 2) + center_offset * np.sin(rad_angle)

        # Append the calculated coordinates to the lists
        src_x_coords.append(src_x)
        src_y_coords.append(src_y)

        # Get the current projection and compute its 1D Fourier Transform
        projection = input_proj[idx, :num_points]
        transformed_projection = np.fft.fftshift(np.fft.fft(np.fft.ifftshift(projection)))

        # Store the Fourier transform values
        fft_values.append(transformed_projection)

    # Combine all coordinates and Fourier transform values into flat arrays
    src_x_coords = np.concatenate(src_x_coords)
    src_y_coords = np.concatenate(src_y_coords)
    fft_values = np.concatenate(fft_values)

    # Create a grid for the output image coordinates
    grid_x, grid_y = np.meshgrid(np.arange(output_shape[0]), np.arange(output_shape[0]))
    grid_x_flat = grid_x.flatten()
    grid_y_flat = grid_y.flatten()

    # Interpolate the Fourier transform values onto the output grid
    reconstructed_fft = scipy.interpolate.griddata(
        (src_x_coords, src_y_coords),
        fft_values,
        (grid_x_flat, grid_y_flat),
        method='linear',
        fill_value=0.0
    ).reshape(output_shape)

    return reconstructed_fft

In [ ]:
res = reconstruction2(projs, step_size, shape)

In [ ]:
spec_mod = np.log(abs(res))
figure, axis = plt.subplots(1, 2, figsize=(15, 15)) # subplots let you visualize multiple outputs simultanously
axis[0].imshow(spec, cmap='gray')
axis[1].imshow(spec_mod, cmap='gray')

In [ ]:
res2 = np.real(np.fft.fftshift(np.fft.ifft2(np.fft.ifftshift(res))))

show_gray(res2)